In [20]:
import pandas as pd
import time
from tqdm import tqdm
from uuid import uuid4

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
INDEX_NAME = "funeral-services"
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'funeral_facilities': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 2756,
 'vector_type': 'dense'} 




In [22]:
# 각 파일별로 'type' 지정 및 컬럼 매핑 정의
file_configs = [
    {
        "filename": "cemeteries.csv",
        "type": "cemetery", # 묘지
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "colubarium.csv",
        "type": "columbarium", # 봉안당
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "crematoriums.csv",
        "type": "crematorium", # 화장시설
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "natural_burial.csv",
        "type": "natural_burial", # 자연장지
        "mapping": {"지역": "region", "소재지": "location", "시설명": "name", "주소": "address", "연락처": "phone"}
    },
    {
        "filename": "funeral_homes.csv",
        "type": "funeral_home", # 장례식장
        "mapping": {"소재지": "region", "장례식장 이름": "name", "주소": "address", "전화번호": "phone"}
        # 장례식장은 'location'(구/군) 컬럼이 없으므로 주소에서 추출 예정
    }
]

def load_and_unify_data(configs):
    combined_df = pd.DataFrame()
    
    for config in configs:
        try:
            # 파일 읽기 (인코딩 에러 처리)
            try:
                df = pd.read_csv('./data/processed/' + config["filename"])
            except UnicodeDecodeError:
                df = pd.read_csv('./data/processed/' + config["filename"], encoding='cp949')
            
            # 컬럼명 변경
            df = df.rename(columns=config["mapping"])
            
            # type 컬럼 추가
            df['type'] = config["type"]
            
            # 장례식장 파일 등 'location'이 없는 경우 주소에서 추출 (예: "서울 종로구..." -> "종로구")
            if 'location' not in df.columns:
                df['location'] = df['address'].apply(
                    lambda x: x.split()[1] if isinstance(x, str) and len(x.split()) > 1 else ""
                )
            
            # full_region: 주소 앞 2개 단어 추출 (예: "경기도 수원시")
            df['full_region'] = df['address'].apply(
                lambda x: " ".join(x.split()[:2]) if isinstance(x, str) else ""
            )
            
            # 필요한 컬럼만 선택 및 결측치 처리
            target_cols = ['type', 'region', 'location', 'full_region', 'name', 'address', 'phone']
            
            # 데이터프레임에 없는 컬럼이 있다면 빈 값으로 생성
            for col in target_cols:
                if col not in df.columns:
                    df[col] = ""
            
            df = df[target_cols].fillna("정보없음")
            
            # 검색에 사용될 'text' 필드 생성 (임베딩 대상)
            df['text'] = df.apply(
                lambda row: f"시설유형: {row['type']}, 지역: {row['full_region']}, 시설명: {row['name']}, 주소: {row['address']}, 전화번호: {row['phone']}",
                axis=1
            )
            
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"[OK] {config['filename']} 로드 완료 ({len(df)}건)")
            
        except Exception as e:
            print(f"[Error] {config['filename']} 처리 중 오류: {e}")
            
    return combined_df

# ---------------------------------------------------------
# 3. Pinecone 적재 로직
# ---------------------------------------------------------
def upload_to_pinecone(df):
    # Pinecone 초기화
    pc = Pinecone()
    
    # 인덱스 확인 및 생성 (없을 경우)
    if INDEX_NAME not in pc.list_indexes().names():
        pc.create_index(
            name=INDEX_NAME,
            dimension=1536, # OpenAI text-embedding-3-small 기준
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
    
    batch_size = 100
    total_rows = len(df)
    
    print(f"\n총 {total_rows}개의 데이터를 Pinecone에 적재합니다...")
    
    for i in tqdm(range(0, total_rows, batch_size)):
        batch = df.iloc[i:i+batch_size]
        
        # 1. 텍스트 임베딩 생성
        texts = batch['text'].tolist()
        ids = [str(uuid4()) for _ in range(len(batch))]
        embeds = embeddings.embed_documents(texts)
        
        # 2. 메타데이터 준비 (full_region 포함)
        # Pinecone은 메타데이터 값을 문자열, 숫자, 불리언, 리스트만 허용함
        metadatas = batch[['type', 'full_region', 'name', 'address', 'phone']].to_dict(orient='records')
        
        # text 필드도 메타데이터에 포함하여 RAG 시 원문 참조 가능하게 함
        for j, meta in enumerate(metadatas):
            meta['text'] = texts[j]
            
        # 3. 벡터 데이터 구성 (id, vector, metadata)
        vectors = list(zip(ids, embeds, metadatas))
        
        # 4. 업로드
        index.upsert(vectors=vectors, namespace='funeral_facilities')
        
    print("적재 완료!")

# ---------------------------------------------------------
# 실행
# ---------------------------------------------------------
if __name__ == "__main__":
    # 1. 데이터 로드 및 전처리
    unified_df = load_and_unify_data(file_configs)
    
    # 2. 데이터 확인
    print("\n--- 데이터 예시 ---")
    # print(unified_df.head(2))
    display(unified_df)
    # 3. Pinecone 적재 (API 키 설정 후 주석 해제하여 실행)
    upload_to_pinecone(unified_df)

[OK] cemeteries.csv 로드 완료 (524건)
[OK] colubarium.csv 로드 완료 (626건)
[OK] crematoriums.csv 로드 완료 (62건)
[OK] natural_burial.csv 로드 완료 (219건)
[OK] funeral_homes.csv 로드 완료 (1087건)

--- 데이터 예시 ---


,type,region,location,full_region,name,address,phone,text
0,cemetery,전북특별자치도,완주군,전라북도 완주군,(재)호정공원(묘지),전라북도 완주군 화산면 운곡로 381-64 (운곡리),063-214-1009,"시설유형: cemetery, 지역: 전라북도 완주군, 시설명: (재)호정공원(묘지)..."
1,cemetery,경상북도,경주시,경상북도 경주시,(재)대명공원묘원,경상북도 경주시 강동면 비학로 50-126 (단구리),054-762-3810,"시설유형: cemetery, 지역: 경상북도 경주시, 시설명: (재)대명공원묘원, ..."
2,cemetery,경기도,광주시,경기도 광주시,(재)시안 가족추모공원(매장묘),경기도 광주시 오포안로 17 (문형동),031-766-2894,"시설유형: cemetery, 지역: 경기도 광주시, 시설명: (재)시안 가족추모공원..."
3,cemetery,전북특별자치도,정읍시,전라북도 정읍시,정읍시입암묘지공원,전라북도 정읍시 입암면 연월리 산159,063-539-7153,"시설유형: cemetery, 지역: 전라북도 정읍시, 시설명: 정읍시입암묘지공원, ..."
4,cemetery,경상북도,김천시,경상북도 김천시,천주교 김천공원묘원,경상북도 김천시 개령면 황계길 280-41 (황계리),054-433-3880,"시설유형: cemetery, 지역: 경상북도 김천시, 시설명: 천주교 김천공원묘원,..."
...,...,...,...,...,...,...,...,...
2513,funeral_home,경남,거창군,경상남도 거창군,거창적십자병원장례식장,"경상남도 거창군 거창읍 중앙로 91 (상림리, 거창적십자병원)",055-944-4482,"시설유형: funeral_home, 지역: 경상남도 거창군, 시설명: 거창적십자병원..."
2514,funeral_home,경남,거창군,경상남도 거창군,거창서경병원장례식장( 거창한국병원장례식장),"경상남도 거창군 거창읍 송정1길 24-13 (송정리, 서경병원)",055-945-0130,"시설유형: funeral_home, 지역: 경상남도 거창군, 시설명: 거창서경병원장..."
2515,funeral_home,경남,합천군,경상남도 합천군,합천추모공원장례식장,경상남도 합천군 합천읍 합천호수로 1613 (합천리),055-934-4444,"시설유형: funeral_home, 지역: 경상남도 합천군, 시설명: 합천추모공원장..."
2516,funeral_home,경남,합천군,경상남도 합천군,합천고려병원 장례식장,경상남도 합천군 대양면 대야로 737-20 (정양리),055-931-4464,"시설유형: funeral_home, 지역: 경상남도 합천군, 시설명: 합천고려병원 ..."



총 2518개의 데이터를 Pinecone에 적재합니다...


100%|██████████| 26/26 [00:57<00:00,  2.23s/it]

적재 완료!


In [23]:
import json

def extract_facilities_region_list(df):
    """type별 지역 리스트 추출"""
    region_dict = {}
    
    for facility_type in df['type'].unique():
        # full_region만 추출하고 정렬
        regions = sorted(df[df['type'] == facility_type]['full_region'].dropna().unique())
        region_dict[facility_type] = regions
    
    return region_dict

# 실행
facilities_regions = extract_facilities_region_list(unified_df)

# JSON 저장
with open('./data/processed/facilities_region_list.json', 'w', encoding='utf-8') as f:
    json.dump(facilities_regions, f, ensure_ascii=False, indent=2)

print("facilities_region_list.json 생성 완료!")
print(json.dumps(facilities_regions, ensure_ascii=False, indent=2))

facilities_region_list.json 생성 완료!
{
  "cemetery": [
    "강원도 태백시",
    "강원특별자치도 강릉시",
    "강원특별자치도 고성군",
    "강원특별자치도 동해시",
    "강원특별자치도 삼척시",
    "강원특별자치도 속초시",
    "강원특별자치도 양구군",
    "강원특별자치도 양양군",
    "강원특별자치도 영월군",
    "강원특별자치도 원주시",
    "강원특별자치도 인제군",
    "강원특별자치도 정선군",
    "강원특별자치도 철원군",
    "강원특별자치도 춘천시",
    "강원특별자치도 평창군",
    "강원특별자치도 홍천군",
    "강원특별자치도 화천군",
    "강원특별자치도 횡성군",
    "경기 파주시",
    "경기도 가평군",
    "경기도 고양시",
    "경기도 광주시",
    "경기도 구리시",
    "경기도 김포시",
    "경기도 남양주시",
    "경기도 동두천시",
    "경기도 성남시",
    "경기도 시흥시",
    "경기도 안산시",
    "경기도 안성시",
    "경기도 양주시",
    "경기도 양평군",
    "경기도 여주시",
    "경기도 연천군",
    "경기도 오산시",
    "경기도 용인시",
    "경기도 의왕시",
    "경기도 의정부시",
    "경기도 이천시",
    "경기도 파주시",
    "경기도 평택시",
    "경기도 포천시",
    "경기도 화성시",
    "경상남도 거제시",
    "경상남도 거창군",
    "경상남도 고성군",
    "경상남도 김해시",
    "경상남도 남해군",
    "경상남도 사천시",
    "경상남도 양산시",
    "경상남도 의령군",
    "경상남도 진주시",
    "경상남도 창원시",
    "경상남도 함안군",
    "경상남도 함양군",
    "경상북도 경산시",
    "경상북도 경주시",
    "경상북

In [ ]:
# for i, row in unified_df.iterrows():
#     print(" ".join(row.address.split()[:2]))

전라북도 완주군
경상북도 경주시
경기도 광주시
전라북도 정읍시
경상북도 김천시
경상남도 양산시
경기도 여주시
인천광역시 서구
제주특별자치도 서귀포시
경기도 성남시
제주특별자치도 서귀포시
대구광역시 북구
인천광역시 옹진군
인천광역시 강화군
경기도 포천시
전라남도 나주시
인천광역시 강화군
경상남도 김해시
경기도 가평군
경기도 포천시
경기도 고양시
충청남도 당진시
강원특별자치도 춘천시
경기도 파주시
경기도 양주시
경기도 양주시
전라남도 영암군
경상남도 거창군
경기도 포천시
경기도 양주시
경상북도 성주군
전라남도 여수시
대전광역시 서구
경상남도 창원시
경기도 의왕시
제주특별자치도 제주시
인천광역시 강화군
인천광역시 강화군
인천광역시 강화군
광주광역시 남구
경기도 김포시
전라북도 전주시
경기도 파주시
경기도 포천시
인천광역시 강화군
대구광역시 군위군
인천광역시 강화군
광주광역시 북구
전라남도 나주시
강원특별자치도 철원군
전라북도 고창군
강원특별자치도 인제군
경기도 파주시
인천광역시 강화군
강원특별자치도 양양군
경기도 용인시
경기도 남양주시
부산광역시 기장군
경상남도 창원시
경상남도 거창군
경기도 안산시
경상북도 김천시
경기도 포천시
강원특별자치도 강릉시
제주특별자치도 제주시
경기도 파주시
충청남도 홍성군
충청북도 충주시
경기도 파주시
전라남도 완도군
경기도 파주시
인천광역시 강화군
경상북도 칠곡군
경상북도 구미시
제주특별자치도 제주시
충청북도 옥천군
강원특별자치도 고성군
인천광역시 강화군
전라북도 김제시
경기도 파주시
부산광역시 금정구
충청북도 충주시
충청남도 예산군
전라북도 장수군
전라북도 익산시
부산광역시 기장군
제주특별자치도 서귀포시
제주특별자치도 서귀포시
경기도 광주시
경기도 용인시
충청남도 논산시
전라남도 나주시
경기도 양주시
충청남도 천안시
강원특별자치도 영월군
인천광역시 강화군
충청남도 부여군
경기도 파주시
경상북도 성주군
경기도 의왕시
인천광역시 강화군
인천광역시 옹진군
전라북도 고창군
경기도 양주시
강원특별자치도 고성군
경기도 여주시
경기도 포천시
전라

In [25]:
unified_df['text']

0       시설유형: cemetery, 지역: 전라북도 완주군, 시설명: (재)호정공원(묘지)...
1       시설유형: cemetery, 지역: 경상북도 경주시, 시설명: (재)대명공원묘원, ...
2       시설유형: cemetery, 지역: 경기도 광주시, 시설명: (재)시안 가족추모공원...
3       시설유형: cemetery, 지역: 전라북도 정읍시, 시설명: 정읍시입암묘지공원, ...
4       시설유형: cemetery, 지역: 경상북도 김천시, 시설명: 천주교 김천공원묘원,...
                              ...                        
2513    시설유형: funeral_home, 지역: 경상남도 거창군, 시설명: 거창적십자병원...
2514    시설유형: funeral_home, 지역: 경상남도 거창군, 시설명: 거창서경병원장...
2515    시설유형: funeral_home, 지역: 경상남도 합천군, 시설명: 합천추모공원장...
2516    시설유형: funeral_home, 지역: 경상남도 합천군, 시설명: 합천고려병원 ...
2517    시설유형: funeral_home, 지역: 경상남도 합천군, 시설명: (주)합천장례...
Name: text, Length: 2518, dtype: object